## Aritificial Neural Networks - ANN

In [2]:
#CARGA DE DATOS Y SEPARACION PREDICTORS-TARGET

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Parte 1 - Pre procesado de datos

### Codificar datos categóricos

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

#codificar variables categóricas en variables dummies (ML-A-Z)
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
)
X = onehotencoder.fit_transform(X)
X = X[:, 1:]

### Dividir el data set en conjunto de entrenamiento y conjunto de testing

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de variables

In [6]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Parte 2 - Construir la RNA

### Importar Keras y librerías adicionales


In [ ]:
import keras
from keras.models import Sequential #iniciarlizar los parámetros de la ANN
from keras.layers import Dense #declarar y crear cada capa intermedia de la ANN / conexión entre capas

In [ ]:
# Cell: 1
import os 
!pip install -U -q segmentation-models --user
os.kill(os.getpid(), 9)

#Cell: 2
import os 
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

### Inicializar la RNA

In [ ]:
classifier = Sequential()

### Añadir las capas de entrada y primera capa oculta


In [ ]:
#experimentación no teorica
#regla oro: capa oculta: media entre capa de entrada y capa de salida mean(11,1)=6
#funcion activación: experimentación: rectificador lineal unitario
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))
#units: numero de nodos
#kernel: pesos iniciales de los coef. - uniforme/constante/....
#activation: en función de los datos de salida / activar capas intermedias, si la información es relevante se activa
#input_dim: tamaño de entrada de los datos (variables)

### Añadir la segunda capa oculta

In [ ]:
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

### Añadir la capa de salida

In [ ]:
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
#sigmoid: devuelve entre 0 y 1 (si se cumple la hipótesis o no)
#si queremos clasificar en n categorias: units=n, activation="escalon o relu"

### Compilar la RNA

In [ ]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
#optimizer: algoritmo de conjunto optimo de pesos: grad. desc. / estocastico/ adam (por defecto y recomendado)
#loss: funcion de perdidas: diferencias cuadrados distancias (num salida), crossentropy (si es binaria o categórica), ...
#metrics: precisión TP, FP, TN, FN // en regresión deberia ser RMSE??

### Ajustamos la RNA al Conjunto de Entrenamiento

In [ ]:
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)
#batch_size: tamaño del bloque a procesar
#epochs: número de iteraciones globales

# Parte 3 - Evaluar el modelo y calcular predicciones finales

### Predicción de los resultados con el Conjunto de Testing

In [ ]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

### Elaborar una matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)